<a href="https://colab.research.google.com/github/Dias-lezdo/Meta-Lllama-Llama-3.3-70B-Instruct-task/blob/main/Meta_Lllama_Llama_3_3_70B_Instruct_using_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -q groq PyPDF2

In [14]:
import json
from groq import Groq
import PyPDF2
client = Groq(api_key="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")

In [15]:
def pdf_to_pagewise_text(pdf_path, output_text_path):
    """Extract pagewise data from a PDF and save it to a text file."""
    try:
        with open(pdf_path, 'rb') as file:
            # Create PDF reader object
            reader = PyPDF2.PdfReader(file)
            with open(output_text_path, "w") as output_file:
                for page_num, page in enumerate(reader.pages, start=1):
                    # Extract text from the current page
                    text = page.extract_text().strip()
                    if text:  # Skip empty pages
                        output_file.write(f"Page {page_num}:\n")
                        output_file.write(text + "\n\n")
            print(f"Pagewise text saved to {output_text_path}")
    except Exception as e:
        print(f"Error reading PDF: {e}")

# Path to your PDF file
pdf_path = "/content/Morgan_ Tujuanza - OCR_removed 10page.pdf"

# Path to save the extracted text
output_text_path = "Morgan.txt"

# Extract pagewise data and save to text file
pdf_to_pagewise_text(pdf_path, output_text_path)

Pagewise text saved to Morgan.txt


In [ ]:
import json
import torch

def chunk_text(text, chunk_size, overlap):
    """Split text into overlapping chunks."""
    tokens = text.split()
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = " ".join(tokens[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(tokens):
            break
    return chunks

def process_chunk(chunk, client):
    """Process a single chunk using the RAG prompt logic."""
    rag_prompt = (
        "The following text is retrieved from a document, indexed by pages in chronological order:\n\n"
        f"{chunk}\n\n"
        "Document Content:\n\n"
        "Your Task:\n"
        "1. Analyze the document content to perform the following tasks:\n"
        "   - Group the content by **evaluation date** ('Date of Service') and **Title**. Entries with the same 'Date of Service' and 'Title' must be consolidated into a single entry.\n"
        "   - Consolidate page numbers into a single range (e.g., 1-3) for grouped entries. Ensure that entries with different 'Date of Service' values or 'Title' are not combined, even if their content appears sequentially.\n"
        "   - Ignore any page numbers mentioned inside the 'Details' field and focus only on the 'Page Number' field for grouping.\n"
        "   - Include only the relevant content and exclude sections such as **Assessment**, **Plan**, **Review of Systems**, and **Physical Examination** from visit types.\n"
        "2. Determine the **Facility/Provider** based on the following rules, ensuring accuracy and consistency:\n"
        "   - If a **signed provider name** exists at the bottom of the page, treat it as the 'Treating Provider.'\n"
        "   - If both a signed provider and an attending provider are present on the page, prioritize the signed provider **only if they are explicitly indicated as the treating provider**. If there is ambiguity, include both providers, but list the signed provider first.\n"
        "   - Cross-check the signed provider at the bottom of the page with the provider name at the top of the page for consistency. If the names match, use the signed provider. If they do not match, treat the signed provider as the default unless there's a clear indication otherwise.\n"
        "   - If no signed provider exists at the bottom of the page, use the provider name mentioned at the top of the page, if available.\n"
        "   - If no provider name exists on the page (neither at the top nor signed at the bottom), mark the provider as 'Unknown provider.'\n"
        "   - Format the provider's name with their designation (e.g., 'Dr. B Bandhu, M.D.') and the complete center name (e.g., 'Georgia Spine and Orthopaedics').\n"
        "3. Collect the **Specialty** of the provider if explicitly mentioned in the document. If not mentioned, give *unknown*.\n"
        "4. Capture the following information for each grouped entry:\n"
        "   - **Date of Service**: The date of the visit, surgery, or progress note.\n"
        "   - **Facility/Provider**: The determined provider name and center based on the above logic.\n"
        "   - **Title**: Identify the type of report from the content (e.g., Consultation, Surgery, Progress Note).\n"
        "   - **Specialty**: The specialty of the provider, if available.\n"
        "   - **Page Number**: Consolidate the page numbers from the 'Page Number' field in the JSON into a range (e.g., 1-3), ensuring that entries with different 'Date of Service' or 'Title' are not combined.\n"
        "5. Validation Rules:\n"
        "   - Ensure that:\n"
        "       * All dates are formatted as MM/DD/YYYY.\n"
        "       * No textual or non-standard date formats are used.\n"
        "       * Do not hallucinate or infer content not explicitly mentioned in the source document.\n"
        "6. Chronological and Logical Organization:\n"
        "   - Reorganize shuffled pages strictly within the JSON-provided page range.\n"
        "   - Arrange reports chronologically based on the Date of Service (DOS).\n"
        "   - If dates are unavailable or illegible, maintain the listed order from the JSON key.\n"
        "7. Deduplication Rules:\n"
        "   - Identify and remove duplicate entries based on the following fields:\n"
        "       * 'Date of Service'\n"
        "       * 'Facility/Provider'\n"
        "       * 'Title'\n"
        "       * 'Specialty'\n"
        "   - If entries match across all fields, merge their page numbers into a single range (e.g., '1-3') and keep only one consolidated entry.\n"
        "8. Format the output into a JSON structure with the following schema:\n"
        "[\n"
        "    {\n"
        "        \"Date of Service\": \"MM-DD-YYYY\",\n"
        "        \"Facility/Provider\": \"Provider Name - Center Name\",\n"
        "        \"Title\": \"Type of Report\",\n"
        "        \"Specialty\": \"Specialty\",\n"
        "        \"Page Number\": \"Page Range\"\n"
        "    }\n"
        "]\n"
        "Respond only with the JSON. Do not provide any additional text or reasoning.\n"
    )


    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an intelligent assistant tasked with answering queries by strictly analyzing the provided context from a document."},
            {"role": "user", "content": rag_prompt}
        ],
        model="llama-3.3-70b-versatile",
        temperature=0.6,
        max_tokens=6000,
        top_p=1,
        stop=None,
        stream=False,
    )

    return json.loads(chat_completion.choices[0].message.content)

def process_pagewise_data(pagewise_data, client, chunk_size=512, overlap=128):
    """Process the pagewise data in chunks."""
    chunks = chunk_text(pagewise_data, chunk_size, overlap)
    final_results = []

    print(f"Total chunks: {len(chunks)}")
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i + 1}/{len(chunks)}...")
        torch.cuda.empty_cache()
        chunk_result = process_chunk(chunk, client)
        final_results.extend(chunk_result)

    return final_results

# Read text from the file
with open("/content/Morgan.txt", "r") as file:
    pagewise_data = file.read()

# Assuming `client` is defined and initialized
final_json_results = process_pagewise_data(pagewise_data, client)

# Save or print the results
print(json.dumps(final_json_results, indent=4))



**OUTPUT FORMAT**

[

{
        "Date of Service": "05/12/2022",
        "Facility/Provider": "Erik Bendiks, MD - Georgia Spine and Orthopaedics",
        "Title": "Progress Note",
        "Specialty": "Orthopaedics",
        "Page Number": "1-2"
    },
    {
        "Date of Service": "05/11/2022",
        "Facility/Provider": "Erik T Bendiks MD - Georgia Spine and Orthopaedics",
        "Title": "Progress Note",
        "Specialty": "unknown",
        "Page Number": "9-10"
    },
]